## Imports

In [2]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import random

from matplotlib.colors import LinearSegmentedColormap
# from selenium import webdriver
# from selenium.webdriver.chrome.options import Options
# from selenium.webdriver.common.by import By
# from selenium.webdriver.support import expected_conditions as ec
# from selenium.webdriver.support.ui import Select
# from selenium.webdriver.support.ui import WebDriverWait
# from webdriver_manager.chrome import ChromeDriverManager

In [3]:
def fetch_strokes_gained():
    """
    Get the expected number of strokes to complete the hole from golfity.com/strokes-gained-calculator for a given surface and distance,
    """
    
    # Launch Chrome browser
    options = Options()
    options.add_argument("--headless")
    driver = webdriver.Chrome(ChromeDriverManager().install(), options=options)
    wait = WebDriverWait(driver, 5)
    driver.get('https://www.golfity.com/strokes-gained-calculator')

    # Initialize empty list to store lists of surface, distance, and expected strokes
    xstrokes = []
    for surface in ['Tee', 'Fairway', 'Rough', 'Sand', 'Green']:

        # Click new surface
        dropdown = Select(driver.find_element(by=By.XPATH, value='/html/body/div[2]/div/div/div[1]/div/form/div[1]/select'))
        dropdown.select_by_visible_text(surface)

        # The max distance for "Green" is 119, for all other surfaces the max distance is 600
        if surface == "Green":
            max_distance = 120
        else:
            max_distance = 601

        # Fetch the expected number of strokes for every whole number distance from 1 to max
        for distance in range(1, max_distance):

            # Enter the distance
            wait.until(ec.visibility_of_element_located((By.XPATH, '/html/body/div[2]/div/div/div[1]/div/form/div[1]/input')))
            driver.find_element(by=By.XPATH, value='/html/body/div[2]/div/div/div[1]/div/form/div[1]/input').clear()
            inputElement = driver.find_element(by=By.XPATH, value="/html/body/div[2]/div/div/div[1]/div/form/div[1]/input")
            inputElement.send_keys(f"{distance}")

            # Store the expected strokes
            benchmark = driver.find_element(by=By.XPATH, value='/html/body/div[2]/div/div/div[1]/div/form/div[1]/div').text
            xstrokes.append([surface, distance, benchmark])

    # Turn the distance into a float and return dataframe
    xstrokes_df = pd.DataFrame(xstrokes, columns=['surface', 'distance', 'benchmark'])
    xstrokes_df['xstrokes'] = xstrokes_df['benchmark'].str.split(': ').str[1].str.split(' ').str[0]
    xstrokes_df['xstrokes'] = xstrokes_df['xstrokes'].astype('float')
    xstrokes_df.drop('benchmark', axis=1, inplace=True)
    
    return xstrokes_df

In [4]:
def simulate_stroke_probs(xstrokes, n_trials):
    """
    Using a normal distribution, model likelihood of n strokes to complete the hole given a distance
    """
    
    # Initiate empty lists to store probabilities of finishing in n strokes from each distance
    probs_1 = []
    probs_2 = []
    probs_3 = []
    probs_4 = []
    probs_5 = []
    probs_6 = []
    probs_7 = []

    # Iterate through each row, model the expected number of strokes given a distance
    for ix, row in xstrokes.iterrows():

        # Model is a normal distribution with mean=xstrokes and stdev=SQRT(strokes)/strokes)
        scores = []

        # Randomly sample many times and count the frequency of each pull
        for i in range(n_trials):
            scores.append(round(np.random.normal(loc=row['xstrokes'], scale=np.sqrt(row['xstrokes'])/row['xstrokes'])))

        # Turn the frequency into a decimal
        probs_1.append(scores.count(1) / n_trials)
        probs_2.append(scores.count(2) / n_trials)
        probs_3.append(scores.count(3) / n_trials)
        probs_4.append(scores.count(4) / n_trials)
        probs_5.append(scores.count(5) / n_trials)
        probs_6.append(scores.count(6) / n_trials)
        probs_7.append(len([score for score in scores if score >= 7]) / n_trials)

    # Store as new columns
    xstrokes['prob_1'] = probs_1
    xstrokes['prob_2'] = probs_2
    xstrokes['prob_3'] = probs_3
    xstrokes['prob_4'] = probs_4
    xstrokes['prob_5'] = probs_5
    xstrokes['prob_6'] = probs_6
    xstrokes['prob_7'] = probs_7
    
    return xstrokes

In [5]:
def calculate_hole_outcome_probs(xstrokes, strokes_hero, dist_hero, strokes_villain, dist_villain):
    """
    Calculate the probability of winning the hole, tying the hole, and losing the hole given the current state of a hole
    """
    
    # Get the highest number of additional strokes in the xstrokes document
    prob_cols = [x for x in list(xstrokes.columns) if x.startswith('prob')]
    max_strokes = max([int(x.split('_')[1]) for x in prob_cols])
    
    # Get the probabilities of n additional strokes given the distance for hero and villain
    xstrokes_hero = xstrokes[xstrokes['distance'] == dist_hero].copy()
    xstrokes_villain = xstrokes[xstrokes['distance'] == dist_villain].copy()
    
    # Initialize probabilities at 0
    win_prob_hero = 0
    tie_prob = 0
    win_prob_villain = 0
    
    # Initialize empty dictionaries
    scores_hero = dict()
    scores_villain = dict()
    
    # Calculate the probability for the hero's total strokes
    for i in range(1, max_strokes + 1):
        scores_hero[i+strokes_hero] = xstrokes_hero[f"prob_{i}"].values[0]
    
    # Calculate the probability for the villain's total strokes
    for i in range(1, max_strokes + 1):
        scores_villain[i+strokes_villain] = xstrokes_villain[f"prob_{i}"].values[0]
    
    # For every combination of hero score and villain score, add the joint probability to the result probability
    for score_hero, prob_hero in scores_hero.items():
        for score_villain, prob_villain in scores_villain.items():
            if score_hero < score_villain:
                win_prob_hero += prob_hero*prob_villain
                
            elif score_hero > score_villain:
                win_prob_villain += prob_hero*prob_villain
                
            elif score_hero == score_villain:
                tie_prob += prob_hero*prob_villain        
    
    return win_prob_hero, tie_prob, win_prob_villain

In [6]:
def generate_hole_list(n_par_3, n_par_5):
    """
    Create an artificial course with normally sampled distances based on par
    """
    # Initialize the number of holes for a given par
    n_par_4 = 15 - n_par_3 - n_par_3

    # Shuffle the list
    par_list = [3]*n_par_3 + [5]*n_par_5 + [4]*n_par_4
    random.shuffle(par_list)

    # Create dict to store mean hole length for a given par
    hole_dist_dict = dict()
    hole_dist_dict[3] = 165
    hole_dist_dict[4] = 430
    hole_dist_dict[5] = 560

    # Simulate hole lengths, with a max of 600 yards
    hole_list = [min(int(np.random.normal(hole_dist_dict[par], 5*par)), 600) for par in par_list]
    
    return hole_list

In [7]:
def calculate_match_xpoints(xstrokes, hole_list):
    """
    Generate a dictionary to calculate the odds of winning a match at any pre-hole match state
    """

    # Store the number of points for each outcome. Like in hockey, a win is worth 2 points, OT loss is worth 1 point
    win = 2
    lose = 0
    tie = 1.5

    # Create a dictionary to store the outcome probabilities of each hole
    hole_dict = dict()
    for i in range(len(hole_list)):
        hole_dict[i+1] = dict()

        hole_dict[i+1]['win'] = calculate_hole_outcome_probs(xstrokes, 0, hole_list[i], 0, hole_list[i])[0]
        hole_dict[i+1]['tie'] = calculate_hole_outcome_probs(xstrokes, 0, hole_list[i], 0, hole_list[i])[1]
        hole_dict[i+1]['lose'] = calculate_hole_outcome_probs(xstrokes, 0, hole_list[i], 0, hole_list[i])[2]

    # Initialize a dictionary to store expected match points
    match_xpoints_dict = dict()

    # Create a key for every hole
    for i in range(len(hole_list)):
        match_xpoints_dict[i+1] = dict()

    # Get the expected points on hole 15
    match_xpoints_dict[15][1] = win*hole_dict[15]['win'] + win*hole_dict[15]['tie'] + tie*hole_dict[15]['lose']
    match_xpoints_dict[15][0] = win*hole_dict[15]['win'] + tie*hole_dict[15]['tie'] + lose*hole_dict[15]['lose']
    match_xpoints_dict[15][-1] = tie*hole_dict[15]['win'] + lose*hole_dict[15]['tie'] + lose*hole_dict[15]['lose']

    # Store expected points for all guaranteed outcomes
    for i in range(2, 16):
        match_xpoints_dict[15][i] = win
        match_xpoints_dict[15][-i] = lose

    # Get the expected points for every hole/score combination 
    for hole_num in range(14, 0, -1):
        for score in range(-15, 16):
            match_xpoints_dict[hole_num][score] = match_xpoints_dict[hole_num+1][min(score+1, 15)]*hole_dict[hole_num]['win'] + match_xpoints_dict[hole_num+1][score]*hole_dict[hole_num]['tie'] + match_xpoints_dict[hole_num+1][max(score-1, -15)]*hole_dict[hole_num]['lose']


    return match_xpoints_dict

In [16]:
def hammer(xstrokes, match_xpoints, match_hole, match_score, strokes_hero, dist_hero, strokes_villain, dist_villain):
    """
    Calculate the difference in expected points for accepting the hammer and declining the hammer
    """
    hole_win_prob, hole_tie_prob, hole_lose_prob = calculate_hole_outcome_probs(xstrokes, strokes_hero, dist_hero, strokes_villain, dist_villain)
    
    # Calculate the expected match points using a weighted average of expected match points on the next hole with a score delta +2, 0, -2 using hole outcome probabilities as weights
    match_xpoints_hammer_accepted = match_xpoints[match_hole+1][match_score-1]
    match_xpoints_hammer_declined = hole_win_prob*match_xpoints[match_hole+1][match_score+2] + hole_tie_prob*match_xpoints[match_hole+1][match_score] + hole_lose_prob*match_xpoints[match_hole+1][match_score-2]
    
    return match_xpoints_hammer_accepted, match_xpoints_hammer_declined

In [8]:
# Read in expected strokes
xstrokes = pd.read_csv("../data/xstrokes.csv")

In [9]:
# Add in modeled result probabilities
xstrokes = simulate_stroke_probs(xstrokes, 10000)

In [10]:
# Generate random course
hole_list = generate_hole_list(3, 3)

In [13]:
# Generate dictionary with expected points for pre-hole match state
match_xpoints = calculate_match_xpoints(xstrokes, hole_list)

In [23]:
# Get the expected match points for accepting and declining a given match state and hole state 
match_hole = 14
match_score = 1

hero_strokes = 0
hero_dist = 420

villain_strokes = 0
villain_dist = 420

hammer(xstrokes, match_xpoints, match_hole, match_score, hero_strokes, hero_dist, villain_strokes, villain_dist)

(1.25871669, 1.5531318269726737)

In [ ]:
# Next steps
# Calculate the value of owning the hammer/not letting your opponents have the hammer